### Caveat of Using Iterators as Function Arguments

When a function requires an iterable for one of its arguments, it will also work with any iterator (since iterators are themselves iterables).

But things can go wrong if you do that!

Let's say we have an iterator that returns a collection of random numbers, and we want, for each such collection, find the minimum amd maximum value:

In [1]:
import random

In [2]:
class Randoms:
    def __init__(self, n):
        self.n = n
        self.i = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        else:
            self.i += 1
            return random.randint(0, 100)

In [3]:
random.seed(0)
l = list(Randoms(10))
print(l)

[49, 97, 53, 5, 33, 65, 62, 51, 100, 38]


Now we can easily find the min and max values:

In [4]:
min(l), max(l)

(5, 100)

But watch what happens if we do this:

In [5]:
random.seed(0)
l = Randoms(10)

In [6]:
min(l)

5

In [7]:
max(l)

ValueError: max() arg is an empty sequence

That's because when `min` ran, it iterated over the **iterator** `Randoms(10)`. When we called `max` on the same iterator, it had already been exhausted - i.e. the argument to max was now empty!

So, be really careful when using iterators!

Here's another more practical example.

Let's go back to our `cars.csv` data file and write some code that will return the car names and MPG - except we also want to return a value indicating the percentage of the car's MPG to the least fuel efficient car in the list.

To do so we will need to iterate over the file twice - once to figure out the largest MPG value, and another time to make the calculation MPG/min_mpg * 100.

Let's just quickly see what our file looks like:

In [8]:
f = open('cars.csv')
for row in f:
    print(row, end='')
f.close()    

Car;MPG;Cylinders;Displacement;Horsepower;Weight;Acceleration;Model;Origin
STRING;DOUBLE;INT;DOUBLE;DOUBLE;DOUBLE;DOUBLE;INT;CAT
Chevrolet Chevelle Malibu;18.0;8;307.0;130.0;3504.;12.0;70;US
Buick Skylark 320;15.0;8;350.0;165.0;3693.;11.5;70;US
Plymouth Satellite;18.0;8;318.0;150.0;3436.;11.0;70;US
AMC Rebel SST;16.0;8;304.0;150.0;3433.;12.0;70;US
Ford Torino;17.0;8;302.0;140.0;3449.;10.5;70;US
Ford Galaxie 500;15.0;8;429.0;198.0;4341.;10.0;70;US
Chevrolet Impala;14.0;8;454.0;220.0;4354.;9.0;70;US
Plymouth Fury iii;14.0;8;440.0;215.0;4312.;8.5;70;US
Pontiac Catalina;14.0;8;455.0;225.0;4425.;10.0;70;US
AMC Ambassador DPL;15.0;8;390.0;190.0;3850.;8.5;70;US
Citroen DS-21 Pallas;0;4;133.0;115.0;3090.;17.5;70;Europe
Chevrolet Chevelle Concours (sw);0;8;350.0;165.0;4142.;11.5;70;US
Ford Torino (sw);0;8;351.0;153.0;4034.;11.0;70;US
Plymouth Satellite (sw);0;8;383.0;175.0;4166.;10.5;70;US
AMC Rebel SST (sw);0;8;360.0;175.0;3850.;11.0;70;US
Dodge Challenger SE;15.0;8;383.0;170.0;3563.;10.0;70;U

In [9]:
def parse_data_row(row):
    row = row.strip('\n').split(';')
    return row[0], float(row[1])

def max_mpg(data):
    # get an iterator for data (which should be an iterable of some kind)
    max_mpg = 0
    for row in data:
        _, mpg = parse_data_row(row)
        if mpg > max_mpg:
            max_mpg = mpg
    return max_mpg

In [10]:
f = open('cars.csv')
next(f)
next(f)
print(max_mpg(f))
f.close()

46.6


In [11]:
def list_data(data, mpg_max):
    for row in data:
        car, mpg = parse_data_row(row)
        mpg_perc = mpg / mpg_max * 100
        print(f'{car}: {mpg_perc:.2f}%')

In [12]:
f = open('cars.csv')
next(f), next(f)
list_data(f, 46.6)
f.close()

Chevrolet Chevelle Malibu: 38.63%
Buick Skylark 320: 32.19%
Plymouth Satellite: 38.63%
AMC Rebel SST: 34.33%
Ford Torino: 36.48%
Ford Galaxie 500: 32.19%
Chevrolet Impala: 30.04%
Plymouth Fury iii: 30.04%
Pontiac Catalina: 30.04%
AMC Ambassador DPL: 32.19%
Citroen DS-21 Pallas: 0.00%
Chevrolet Chevelle Concours (sw): 0.00%
Ford Torino (sw): 0.00%
Plymouth Satellite (sw): 0.00%
AMC Rebel SST (sw): 0.00%
Dodge Challenger SE: 32.19%
Plymouth 'Cuda 340: 30.04%
Ford Mustang Boss 302: 0.00%
Chevrolet Monte Carlo: 32.19%
Buick Estate Wagon (sw): 30.04%
Toyota Corolla Mark ii: 51.50%
Plymouth Duster: 47.21%
AMC Hornet: 38.63%
Ford Maverick: 45.06%
Datsun PL510: 57.94%
Volkswagen 1131 Deluxe Sedan: 55.79%
Peugeot 504: 53.65%
Audi 100 LS: 51.50%
Saab 99e: 53.65%
BMW 2002: 55.79%
AMC Gremlin: 45.06%
Ford F250: 21.46%
Chevy C20: 21.46%
Dodge D200: 23.61%
Hi 1200D: 19.31%
Datsun PL510: 57.94%
Chevrolet Vega 2300: 60.09%
Toyota Corolla: 53.65%
Ford Pinto: 53.65%
Volkswagen Super Beetle 117: 0.00%
AM

Now let's try and put these together:

In [13]:
with open('cars.csv') as f:
    next(f)
    next(f)
    max_ = max_mpg(f)
    print(f'max={max_}')
    list_data(f, max_)

max=46.6


No output from `list_data`!!

That's because when we called `list_data` we had already exhausted the data file in the call to `max_mpg`.

Our only option is to either create the iterator twice:

In [14]:
with open('cars.csv') as f:
    next(f), next(f)
    max_ = max_mpg(f)
    
with open('cars.csv') as f:
    next(f), next(f)
    list_data(f, max_)

Chevrolet Chevelle Malibu: 38.63%
Buick Skylark 320: 32.19%
Plymouth Satellite: 38.63%
AMC Rebel SST: 34.33%
Ford Torino: 36.48%
Ford Galaxie 500: 32.19%
Chevrolet Impala: 30.04%
Plymouth Fury iii: 30.04%
Pontiac Catalina: 30.04%
AMC Ambassador DPL: 32.19%
Citroen DS-21 Pallas: 0.00%
Chevrolet Chevelle Concours (sw): 0.00%
Ford Torino (sw): 0.00%
Plymouth Satellite (sw): 0.00%
AMC Rebel SST (sw): 0.00%
Dodge Challenger SE: 32.19%
Plymouth 'Cuda 340: 30.04%
Ford Mustang Boss 302: 0.00%
Chevrolet Monte Carlo: 32.19%
Buick Estate Wagon (sw): 30.04%
Toyota Corolla Mark ii: 51.50%
Plymouth Duster: 47.21%
AMC Hornet: 38.63%
Ford Maverick: 45.06%
Datsun PL510: 57.94%
Volkswagen 1131 Deluxe Sedan: 55.79%
Peugeot 504: 53.65%
Audi 100 LS: 51.50%
Saab 99e: 53.65%
BMW 2002: 55.79%
AMC Gremlin: 45.06%
Ford F250: 21.46%
Chevy C20: 21.46%
Dodge D200: 23.61%
Hi 1200D: 19.31%
Datsun PL510: 57.94%
Chevrolet Vega 2300: 60.09%
Toyota Corolla: 53.65%
Ford Pinto: 53.65%
Volkswagen Super Beetle 117: 0.00%
AM

or we could read the entire data set into a list first - but of course if the file is huge we will have some potential for running out memory:

In [15]:
with open('cars.csv') as f:
    data = [row for row in f][2:]

or, more simply:

In [16]:
with open('cars.csv') as f:
    data = f.readlines()[2:]

In [17]:
max_ = max_mpg(data)
list_data(data, max_)

Chevrolet Chevelle Malibu: 38.63%
Buick Skylark 320: 32.19%
Plymouth Satellite: 38.63%
AMC Rebel SST: 34.33%
Ford Torino: 36.48%
Ford Galaxie 500: 32.19%
Chevrolet Impala: 30.04%
Plymouth Fury iii: 30.04%
Pontiac Catalina: 30.04%
AMC Ambassador DPL: 32.19%
Citroen DS-21 Pallas: 0.00%
Chevrolet Chevelle Concours (sw): 0.00%
Ford Torino (sw): 0.00%
Plymouth Satellite (sw): 0.00%
AMC Rebel SST (sw): 0.00%
Dodge Challenger SE: 32.19%
Plymouth 'Cuda 340: 30.04%
Ford Mustang Boss 302: 0.00%
Chevrolet Monte Carlo: 32.19%
Buick Estate Wagon (sw): 30.04%
Toyota Corolla Mark ii: 51.50%
Plymouth Duster: 47.21%
AMC Hornet: 38.63%
Ford Maverick: 45.06%
Datsun PL510: 57.94%
Volkswagen 1131 Deluxe Sedan: 55.79%
Peugeot 504: 53.65%
Audi 100 LS: 51.50%
Saab 99e: 53.65%
BMW 2002: 55.79%
AMC Gremlin: 45.06%
Ford F250: 21.46%
Chevy C20: 21.46%
Dodge D200: 23.61%
Hi 1200D: 19.31%
Datsun PL510: 57.94%
Chevrolet Vega 2300: 60.09%
Toyota Corolla: 53.65%
Ford Pinto: 53.65%
Volkswagen Super Beetle 117: 0.00%
AM

We may even write functions that need to iterate more than once over an iterable. For example:

In [18]:
def list_data(data):
    max_mpg = 0
    for row in data:
        _, mpg = parse_data_row(row)
        if mpg > max_mpg:
            max_mpg = mpg
    
    for row in data:
        car, mpg = parse_data_row(row)
        mpg_perc = mpg / max_mpg * 100
        print(f'{car}: {mpg_perc:.2f}%')

But this will not work if we pass an iterator as the argument:

with open('cars.csv') as f:
    next(f)
    next(f)
    list_data(f)

We might want to be more defensive about this in our function, either by raising an exception if the argument is an iterator, or making an iterable from the iterator:

In [19]:
def list_data(data):
    if iter(data) is data:
        raise ValueError('data cannot be an iterator.')
    max_mpg = 0
    for row in data:
        _, mpg = parse_data_row(row)
        if mpg > max_mpg:
            max_mpg = mpg
    
    for row in data:
        car, mpg = parse_data_row(row)
        mpg_perc = mpg / max_mpg * 100
        print(f'{car}: {mpg_perc:.2f}%')

In [20]:
with open('cars.csv') as f:
    next(f)
    next(f)
    list_data(f)

ValueError: data cannot be an iterator.

or this way:

In [21]:
def list_data(data):
    if iter(data) is data:
        data = list(data)
    
    max_mpg = 0
    for row in data:
        _, mpg = parse_data_row(row)
        if mpg > max_mpg:
            max_mpg = mpg
    
    for row in data:
        car, mpg = parse_data_row(row)
        mpg_perc = mpg / max_mpg * 100
        print(f'{car}: {mpg_perc:.2f}%')

In [22]:
with open('cars.csv') as f:
    next(f)
    next(f)
    list_data(f)

Chevrolet Chevelle Malibu: 38.63%
Buick Skylark 320: 32.19%
Plymouth Satellite: 38.63%
AMC Rebel SST: 34.33%
Ford Torino: 36.48%
Ford Galaxie 500: 32.19%
Chevrolet Impala: 30.04%
Plymouth Fury iii: 30.04%
Pontiac Catalina: 30.04%
AMC Ambassador DPL: 32.19%
Citroen DS-21 Pallas: 0.00%
Chevrolet Chevelle Concours (sw): 0.00%
Ford Torino (sw): 0.00%
Plymouth Satellite (sw): 0.00%
AMC Rebel SST (sw): 0.00%
Dodge Challenger SE: 32.19%
Plymouth 'Cuda 340: 30.04%
Ford Mustang Boss 302: 0.00%
Chevrolet Monte Carlo: 32.19%
Buick Estate Wagon (sw): 30.04%
Toyota Corolla Mark ii: 51.50%
Plymouth Duster: 47.21%
AMC Hornet: 38.63%
Ford Maverick: 45.06%
Datsun PL510: 57.94%
Volkswagen 1131 Deluxe Sedan: 55.79%
Peugeot 504: 53.65%
Audi 100 LS: 51.50%
Saab 99e: 53.65%
BMW 2002: 55.79%
AMC Gremlin: 45.06%
Ford F250: 21.46%
Chevy C20: 21.46%
Dodge D200: 23.61%
Hi 1200D: 19.31%
Datsun PL510: 57.94%
Chevrolet Vega 2300: 60.09%
Toyota Corolla: 53.65%
Ford Pinto: 53.65%
Volkswagen Super Beetle 117: 0.00%
AM